# iBridges First Steps

This note book is for the impatient user. We will take you through the main functionality of the iBridges API.

To follow the notebook you need a valid iRODS configuration file installed in your `.irods` directory. If you do not have that please follow our guide [here](01-Setup-and-connect.ipynb).

## Create an iRODS session (connection to iRODS server)

In [ ]:
from ibridges.interactive import interactive_auth
session = interactive_auth()

The `interactive_auth` takes the default `.irods/irods_environment.json` to connect to the iRODS server. You can direct the authentication to a different file like this:

In [ ]:
from pathlib import Path
session = interactive_auth(irods_env_path=Path("~").expanduser().joinpath(".irods", "another_env_file.json"))

### Checking some session parameters

In [ ]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)
print(session.home)

## A word on iRODS paths

On a local file name we have folders/directories and files. In iRODS we call them collections and data objects. In contrast to files, data objects carry system metadata and user defined metadata. Likewise for collections.

`iBridges` supports iRODS paths with an own module `IrodsPath`. In this module all paths will be relative to the `session.home` unless they start with `/`.

There are three ways to create an `IrodsPath` from your home.

In [ ]:
from ibridges import IrodsPath
home = IrodsPath(session, session.home)
home1 = IrodsPath(session, "~")
home2 = IrodsPath(session)

print(home)
print(home1)
print(home2)

`IrodsPaths` implement some functionalities comparable to their counterparts in `Pathlib`.

In [ ]:
# some stats
print(home)
print(home.exists())
print(home.dataobject_exists())
print(home.collection_exists())
print(home.parent)

# path manipulation
print(home.joinpath("sub_collection"))

# iRODS system metadata
print(f"Size of {home}: {home.size}") # size in bytes

# iRODS operations
new_coll = IrodsPath.create_collection(session, home.joinpath("new_coll_name"))
new_coll_path = IrodsPath(session, new_coll.path)
print(f"Created: {new_coll_path}")
new_coll_path = new_coll_path.rename("new_coll_name1")
print(f"Moved to: {new_coll_path}")
new_coll_path.remove()

For a full Tutorial please have a look [here](02-iRODS-paths.ipynb).

## Working with data

[Full Tutorial](03-Working-with-data.ipynb) and a tutorial for [Data Synchronisation](06-Data-sync.ipynb)

In [ ]:
from pathlib import Path
from ibridges.util import obj_replicas
from ibridges import upload, download

### Upload a file or folder

Create a local file:

In [ ]:
# create a local file
local_path = Path.expanduser(Path('~')).joinpath("demofile.txt")
f = open(local_path, "a")
f.write("My content! Super important")
f.close()

Determine the iRODS path the file should be uploaded to.

In [ ]:
irods_path = IrodsPath(session, session.home, "new_coll")
if not irods_path.collection_exists():
    coll = IrodsPath.create_collection(session, irods_path)

We can first check that an upload would change with the `dry-run`:

In [ ]:
# upload the file to our collection
from pprint import pprint
ops = upload(session, local_path, irods_path, dry_run=True, overwrite=True)
pprint(ops)

The function will only show what really needs to be updated on iRODS based on checksums. Now we can really upload the file. We will use the `overwrite` flag to update all existing data in iRODS.

In [ ]:
upload(session, local_path, irods_path, overwrite=True)

The `ops` dictionary can also be used to log what has been uploaded.

### Retrieve a  data object from iRODS and checking their status

In [ ]:
obj_path = irods_path.joinpath("demofile.txt")
obj = obj_path.dataobject

Some system metadata of the object:

In [ ]:
print('data object name\t', obj.name)
print('data object path\t', obj.path)
print('data object size\t', obj.size)
print('data object checksum\t', obj.checksum)
print('data replicas\t', obj_replicas(obj))

### Open a data object in read or write mode

In [ ]:
stream = obj.open('r')
text = stream.read().decode()
stream.close()
print(text)

### Download a data object or collection

Downloads work similar as the uploads: use the `dry_run` to see what will change and use the `overwrite` flag to update all already existing data.

In [ ]:
ipath = IrodsPath(session, irods_path)
print(ipath)

In [ ]:
from pathlib import Path
local_path = Path("~/Downloads").expanduser()
print("Download folder exists", local_path.is_dir())
download(session, ipath, local_path, overwrite=True)

## User defined metadata of data objects and collections

[Full Tutorial](04-Metadata.ipynb)

In [ ]:
# upload a file
local_path = Path("~").expanduser().joinpath("demofile.txt")
irods_path = IrodsPath(session, session.home, "new_coll")
if not irods_path.collection_exists():
    coll = IrodsPath.create_collection(session, irods_path)
upload(session, local_path, irods_path, overwrite = True)

### Retrieve an iRODS object or collection and list its metadata

In [ ]:
obj = irods_path.joinpath("demofile.txt").dataobject

In [ ]:
from ibridges.meta import MetaData
obj_meta = MetaData(obj)
print(obj_meta)

### View, add, set and delete metadata

In [ ]:
obj_meta.add('NewKey', 'NewValue')
obj_meta.add('NewKey', 'AnotherValue')
print(obj_meta)

In [ ]:
obj_meta.delete('NewKey', 'NewValue')
print(obj_meta)

We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [ ]:
obj_meta.set("NewKey", "YetAnotherValue")
print(obj_meta)

### Accessing metadata 
With the print function you can quickly inspect the metadata of an iRODS collection or object. If you want to extract and do something with the metadata, you can iterate over all metadata entries. We give a small example below where we assume that the metadata contains a key/name *AUTHOR*:

In [ ]:
for md in obj_meta:
    print(md.name, "has value", md.value)

## Search for data

How can you retrieve the location of an iRODS collection or data object by its metadata or partial part? To this end we provide the function `search`

In [ ]:
from ibridges.search import search_data

#all collections and objects with 
key_vals = {"NEWKEY": "YetAnotherValue"}
result = search_data(session, key_vals=key_vals)
print(result)

The result gives us the collection, the object name, and its checksum.

The wildcard in iRODS searches is the `%`.

In [ ]:
result = search_data(session, path=session.home+"/new_coll/%")
print(result)

## Permissions

**Note: In Yoda 1.9 users cannot set permissions any longer.**

[Full Tutorial](05-Data-Sharing.ipynb)

### Accessing the permissions of a data object or collection in iRODS

Objects and collections have permissions attached to them. Permissions, which work like access levels, must be specified per user or group. The basic permissions are `own` (implies reading and writing), `modify object` (editing and reading), and `read object`.

In [ ]:
from ibridges.permissions import Permissions

# select a file to inspect and set permissions on
item_path = IrodsPath(session, session.home) # Path to collection or data object
collection = item_path.collection  # Get the collection, use item_path.dataobject for data objects

# instantiate permissions with that object
perm = Permissions(session, collection)
print(f'Permissions for {item_path}:\n')
print(perm)

### Available permissions on your iRODS server

In [ ]:
perm.available_permissions

### Adding permissions to a collection or data object

In [ ]:
perm.set('modify object', '<username or group name>')
print(perm)

Note that some permission-types have synonyms:

+ read object: 'read', 'read object', 'read_object'
+ modify object: 'write', 'modify object', 'modify_object'

### Removing permissions

In [ ]:
perm.set('null', '<username or group name>')
print(perm)

### Inheritance

Collections have two special permissions level `inherit` and `noinherit`. From the point in time where inheritance in switched on, all newly added subcollections and data objects will inherit their initial permissions from the collection.

In [ ]:
# Retrieve a collection from iRODS
coll_path = session.home
coll = session.irods_session.collections.get(item_path)
coll_perm = Permissions(session, coll)

#Switch inheritance on
coll_perm.set('inherit')
print(coll_perm)

In [ ]:
# Switch inheritance off
coll_perm.set('noinherit')
print(coll_perm)